In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import MeCab
from pandas.core.frame import DataFrame
from gensim.models import KeyedVectors
from gensim import models
from tqdm import tqdm

ModuleNotFoundError: No module named 'MeCab'

In [7]:
feature = pd.read_csv('feature_des.csv', sep='\t')

In [8]:
feature

,deal_id,deal_category_cd,shop_id,deal_type_cd,deal_price,discount_rate,deal_title,month,duration_days
0,112440,3,13869,1,3200,64,【10本アートし放題／フット利用可／2枚利用可／オフ込み／駐車場有】JNE1級／JNA上級取...,12,14
1,112441,8,10029,2,3130,0,Classic Backpack／DARK RED,12,7
2,112442,2,10479,2,3240,0,【送料込み】キャベツやニラを入れると際立つもつの旨み。家族団らんにもぴったり《博多もつ鍋》お...,12,7
3,112443,8,10029,2,3130,0,Classic Backpack／BRT BLUE,12,7
4,112444,9,8334,2,1400,35,【2色展開】水出し茶や自家製サングリアを手軽に楽しめる。スタイリッシュ＆レトロなワインボトル...,12,14
5,112445,8,10029,2,3130,0,Classic Backpack／PINK GLO,12,7
6,112446,9,8334,2,5480,37,【安心の日本製】ご飯の炊ける様子がよく見える。耐熱性に優れたシックな炊飯土釜＆透明ガラスフタ...,12,14
7,112447,11,6593,2,6800,70,【70％OFF／8畳用】《LEDシーリングライト 調光8畳用》省エネ・長寿命・低コストでお得...,12,12
8,112448,8,10029,2,3130,0,Classic Backpack／TEAL,12,7
9,112449,9,9833,2,3500,0,電子レンジで2分加熱するだけ。ノンオイルでヘルシーなポップコーンを、自宅で手軽に作れるシリコ...,12,12


In [9]:
feature = feature.drop(['deal_id'], axis=1)

In [10]:
title=feature.pop('deal_title')

In [11]:
model = KeyedVectors.load_word2vec_format('entity_vector.model.bin', binary=True)
mecab = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

NameError: name 'KeyedVectors' is not defined

In [6]:
vec_des=[]
for doc in tqdm(title):
    segment = mecab.parse(doc)
    segment_word_list = [line.split('\t')[0] for line in segment.split('\n')][:-2]
    vector = [0]
    counter = 0
    for word in segment_word_list:
        try:
            v = model.wv[word]
            counter += 1
        except:
            continue
        vector = vector + v;
    vec_des.append(np.array(vector)/counter)

100%|██████████| 161690/161690 [00:54<00:00, 2987.94it/s]


In [7]:
temp = DataFrame(vec_des)
temp.rename(columns=lambda x: 'dspt_vec'+x.astype(str), inplace=True)

In [8]:
new_feature = pd.concat([feature, temp], axis=1)

In [9]:
new_feature.to_csv('feature.csv', index=False, sep='\t')